In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, MultiLineString
import os
import osmnx as ox
from itertools import combinations
import random
import numpy as np
import math
import itertools
from igraph import union
import json

In [ ]:
energy_lines_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\OSM\\osm_power_lines_cantabria.shp"
generation_points_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\Combined\\combined_osm_power_cantabria_global_power.shp"
substations_file_path="C:\\Users\\revueltaap\\UNICAN\\EMCAN 2024 A2 ADAPTA - Documentos\\02_Tareas\\Proyecto redes\\DATABASES\\OSM\\osm_power_subest_cantabria.shp"


gdf_cut = ox.geocode_to_gdf("Cantabria, Spain")

In [ ]:
networks_dic = {
                'Energy network':{'lines file paths':{'lines':energy_lines_file_path},
                                  'nodes file paths':{'power sources':generation_points_file_path,'substations':substations_file_path},
                                  'buffer distance':0.01,'buffer option':'to nodes'}
                }

In [ ]:
def combine_gdfs(gdf_cut,paths_dic):
    """
    Une varios shapefiles en un solo GeoDataFrame:
    - reproyectados al CRS del ref_gdf
    - recortados (clip) al ref_gdf
    - con columna extra 'type' = clave del diccionario
    - solo mantiene 'type' + geometry
    """
    gdfs = []

    for key, filepath in paths_dic.items():
        # 1. Leer shapefile
        gdf = gpd.read_file(filepath)

        # 2. Reproyectar al CRS de referencia
        gdf = gdf.to_crs(gdf_cut.crs)

        # 3. Clip con el gdf de referencia
        gdf = gpd.clip(gdf, gdf_cut)

        # 4. Añadir columna con la clave
        gdf["type"] = key

        # 5. Mantener solo la columna 'source' + geometry
        gdf = gdf[["type", "geometry"]]

        gdfs.append(gdf)

    # 6. Concatenar todos en un único GeoDataFrame
    result = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdf_cut.crs)

    return result

In [ ]:
dic=networks_dic['Energy network']
lines_gdf=combine_gdfs(gdf_cut,dic['lines file paths'])
nodes_gdf=combine_gdfs(gdf_cut,dic['nodes file paths'])

In [ ]:
def combine_lines_gdf(gdf):
    # 1. Asegurarse de que todas las geometrías son LineString
    def explode_multilines(gdf):
        out_geoms = []
        out_attrs = []
        for idx, row in gdf.iterrows():
            geom = row.geometry
            if isinstance(geom, LineString):
                out_geoms.append(geom)
                out_attrs.append(row.drop("geometry"))
            elif isinstance(geom, MultiLineString):
                for part in geom.geoms:
                    out_geoms.append(part)
                    out_attrs.append(row.drop("geometry"))
        exploded = gpd.GeoDataFrame(out_attrs, geometry=out_geoms, crs=gdf.crs)
        return exploded

    gdf = explode_multilines(gdf)

    # 2. Construir grafo topológico
    g_nx = nx.Graph()
    for i, line in enumerate(gdf.geometry):
        coords = list(line.coords)
        for j in range(len(coords) - 1):
            p1, p2 = coords[j], coords[j + 1]
            g_nx.add_edge(p1, p2, line_id=i)

    # 3. Encontrar cadenas
    def find_chains(g_nx):
        visited_edges = set()
        chains = []

        for u, v in g_nx.edges():
            if (u, v) in visited_edges or (v, u) in visited_edges:
                continue

            chain = [u, v]
            visited_edges.add((u, v))
            visited_edges.add((v, u))

            # expandir hacia u
            current, prev = u, v
            while g_nx.degree[current] == 2:
                neighbors = [n for n in g_nx.neighbors(current) if n != prev]
                if not neighbors:
                    break
                nxt = neighbors[0]
                chain.insert(0, nxt)
                visited_edges.add((current, nxt))
                visited_edges.add((nxt, current))
                prev, current = current, nxt

            # expandir hacia v
            current, prev = v, u
            while g_nx.degree[current] == 2:
                neighbors = [n for n in g_nx.neighbors(current) if n != prev]
                if not neighbors:
                    break
                nxt = neighbors[0]
                chain.append(nxt)
                visited_edges.add((current, nxt))
                visited_edges.add((nxt, current))
                prev, current = current, nxt

            chains.append(chain)

        return chains

    chains = find_chains(g_nx)

    # 4️⃣ Fusionar geometrías y atributos
    merged_records = []
    for chain in chains:
        merged_line = LineString(chain)

        # índices de líneas originales en la cadena
        line_indices = set()
        for j in range(len(chain) - 1):
            edge_data = g_nx.get_edge_data(chain[j], chain[j + 1])
            line_indices.add(edge_data['line_id'])
        sub_gdf = gdf.iloc[list(line_indices)]

        new_record = {}

        # Caso especial: tunnel
        if "tunnel" in gdf.columns:
            if (sub_gdf["tunnel"] == "yes").any(): # type: ignore
                new_record["tunnel"] = "yes"
            else:
                new_record["tunnel"] = "no"

        # Otros atributos: conservar si son constantes
        for col in gdf.columns:
            if col in ("geometry", "tunnel"):
                continue
            values = sub_gdf[col].dropna().unique()
            if len(values) == 1:
                new_record[col] = values[0]
            else:
                new_record[col] = None  # conflicto → se pierde

        new_record["geometry"] = merged_line
        merged_records.append(new_record)

    merged_gdf = gpd.GeoDataFrame(merged_records, crs=gdf.crs)
    return merged_gdf

lines_gdf=combine_lines_gdf(lines_gdf)

In [ ]:
def gdf_to_nx_buffer_to_nodes(lines_gdf,nodes_gdf,buffer_distance):

    G = nx.Graph()
    coord_to_node = {}
    cross_id = 0

    boundary = gdf_cut.union_all()
    boundary_line = boundary.boundary  # línea del borde

    def merge_extreme_to_station(G, ext_node, station_node):
        """Conecta vecinos del nodo extremo a la estación y elimina el nodo extremo."""
        neighbors = [n for n in G.neighbors(ext_node) if n != station_node]
        for neighbor in neighbors:
            attr = G.get_edge_data(ext_node, neighbor)
            G.add_edge(station_node, neighbor, **attr)
        G.remove_node(ext_node)

    for i, via in lines_gdf.iterrows():
        line_start = Point(via.geometry.coords[0])
        line_end = Point(via.geometry.coords[-1])

        # Nodo inicio
        if tuple(line_start.coords[0]) in coord_to_node:
            start_node = coord_to_node[tuple(line_start.coords[0])]
        else:
            start_node = f"start_{cross_id}"
            G.add_node(start_node, geometry=line_start, type="intersection")
            coord_to_node[tuple(line_start.coords[0])] = start_node
            cross_id += 1

        # Nodo fin
        if tuple(line_end.coords[0]) in coord_to_node:
            end_node = coord_to_node[tuple(line_end.coords[0])]
        else:
            end_node = f"end_{cross_id}"
            G.add_node(end_node, geometry=line_end, type="intersection")
            coord_to_node[tuple(line_end.coords[0])] = end_node
            cross_id += 1

        # Añadir arista que representa la vía
        G.add_edge(start_node, end_node, **via.to_dict())

    G.remove_edges_from(nx.selfloop_edges(G))

    # 1️⃣ Añadir todas las estaciones al grafo
    for idx, est in nodes_gdf.iterrows():
        G.add_node(idx, **est.to_dict())

    # 2️⃣ Iterar sobre nodos extremos válidos
    for n, data in list(G.nodes(data=True)):
        if data.get("type") != "intersection":
            continue
        if G.degree(n) != 1:
            continue

        # Buscar estaciones cercanas
        candidate_stations = [
            idx for idx, est in nodes_gdf.iterrows()
            if est.geometry.distance(data["geometry"]) <= buffer_distance
        ]

        if candidate_stations:
            # Seleccionar la estación más cercana
            nearest_station = min(
                candidate_stations,
                key=lambda idx: nodes_gdf.loc[idx].geometry.distance(data["geometry"])
            )

            # Conectar nodo extremo con estación
            G.add_edge(nearest_station, n, type="train tracks")

            # Hacer merge: la estación reemplaza al nodo extremo
            merge_extreme_to_station(G, n, nearest_station)

    # Obtener nodos extremos (degree == 1)
    end_nodes = [n for n, d in G.degree() if d == 1]

    # Revisar todos los pares de nodos extremos
    for n1, n2 in itertools.combinations(end_nodes, 2):
        if n1 not in G or n2 not in G:
            continue

        geom1 = G.nodes[n1]["geometry"]
        geom2 = G.nodes[n2]["geometry"]

        # Solo procesar si están dentro de la distancia
        if geom1.distance(geom2) <= buffer_distance:
            type1 = G.nodes[n1].get("type")
            type2 = G.nodes[n2].get("type")

            types=['substations','power sources']

            if type1 in types and type2 in types:
                # Caso especial: ambos son estaciones, solo añadir arista entre ellos
                G.add_edge(n1, n2, type="train tracks")
            else:
                geom = G.nodes[n1]["geometry"]
                if geom.distance(boundary_line)<1e-6:
                    neighbors_n2 = list(G.neighbors(n2))
                    for v in neighbors_n2:
                        if n1 == v:
                            continue
                        G.add_edge(n1, v, type="train tracks")
                    G.remove_node(n2)
                else:
                    # Caso normal: merge/fusión de nodos extremos
                    neighbors_n1 = list(G.neighbors(n1))
                    neighbors_n2 = list(G.neighbors(n2))

                    # Conectar todos los vecinos entre sí (evitando los nodos extremos)
                    for u in neighbors_n1:
                        for v in neighbors_n2:
                            if u == v:
                                continue
                            G.add_edge(u, v, type="train tracks")

                    # Eliminar los dos nodos extremos
                    G.remove_node(n1)
                    G.remove_node(n2)



    # Suponemos que gdf_cut es un polígono (o multipolygon) que define los límites

    isolated_nodes = [n for n, d in G.degree() if d == 0]

    for n in isolated_nodes:
        geom_n = G.nodes[n]["geometry"]

        # Buscar el nodo más cercano que tenga grado > 0
        candidate_nodes = [m for m, d in G.degree() if d > 0]
        if not candidate_nodes:
            continue

        nearest_node = min(
            candidate_nodes,
            key=lambda m: geom_n.distance(G.nodes[m]["geometry"])
        )
        geom_nearest = G.nodes[nearest_node]["geometry"]

        # Línea entre nodo aislado y más cercano
        line = LineString([geom_n, geom_nearest])

        if boundary.contains(line):
            # Caso 1: la línea está dentro -> conectar directamente
            G.add_edge(n, nearest_node, type="train tracks")
        else:
            # Caso 2: la línea se sale -> buscar intersección con el límite
            inter = line.intersection(boundary.boundary)

            if not inter.is_empty:
                # Si hay varias intersecciones, cogemos la más cercana al nodo aislado
                if inter.geom_type == "MultiPoint":
                    inter_point = min(inter.geoms, key=lambda p: geom_n.distance(p))
                else:
                    inter_point = inter

                # Crear un nuevo nodo en el punto de salida
                new_node = f"boundary_{n}"
                G.add_node(new_node, geometry=inter_point, type="boundary")

                # Conectar aislado hasta el límite
                G.add_edge(n, new_node, type="train tracks")

    # Obtener lista de componentes conexas
    components = list(nx.connected_components(G))

    checked = set()  # componentes que ya no se pueden conectar

    while len(components) > 1:
        progress = False  # para saber si en esta iteración conectamos algo

        for smallest_comp in components:
            comp_key = tuple(sorted(map(str, smallest_comp)))
            if comp_key in checked:
                continue

            # Todas las otras componentes
            other_comps = [c for c in components if c != smallest_comp]

            # Inicializar variables para la arista más corta
            min_dist = float("inf")
            closest_pair = None

            # Buscar el par de nodos (uno en smallest_comp, otro en otra componente) más cercano
            for n1 in smallest_comp:
                geom1 = G.nodes[n1]["geometry"]
                for comp in other_comps:
                    for n2 in comp:
                        geom2 = G.nodes[n2]["geometry"]
                        dist = geom1.distance(geom2)
                        if dist < min_dist:
                            min_dist = dist
                            closest_pair = (n1, n2)

            # Conectar si está por debajo del umbral
            if closest_pair and min_dist <= 10*buffer_distance:
                n1, n2 = closest_pair
                G.add_edge(
                    n1, n2, type="train tracks")
                progress = True
                break  # volvemos a recalcular componentes tras una conexión
            else:
                # Marcar esta componente como no conectable en este paso
                checked.add(comp_key)

        if not progress:
            # No hemos podido conectar ninguna componente más
            break

        # Recalcular componentes
        components = list(nx.connected_components(G))
        checked.clear()  # reiniciar porque las componentes han cambiado

    # 3️⃣ Recorrer nodos y comprobar si su geometry toca el borde
    for n, data in G.nodes(data=True):
        geom = data.get("geometry")  # debe ser shapely.Point
        if geom.distance(boundary_line)<1e-6:
            #print(n)
            G.nodes[n]["type"] = "boundary"



    return G

g_nx=gdf_to_nx_buffer_to_nodes(lines_gdf,nodes_gdf,0.005)

In [ ]:
def plot_gdf(network,gdf_lines,gdf_nodes,gdf_cut):
    fig, ax = plt.subplots(figsize=(10, 8))
    gdf_cut.plot(ax=ax, color="lightgrey", edgecolor="black", figsize=(8, 8))
    gdf_lines.plot(ax=ax, color="green", edgecolor="black", linewidth=0.5)
    gdf_nodes.plot(ax=ax, color="red", markersize=0.5)

    plt.title("Mapa de "+network)
    file_name="mapa "+network+".png"
    plt.savefig(file_name, dpi=300, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

def plot_ig_graph(network, g_ig, gdf_cut, node_colors,edge_colors):
    # Extraer coordenadas de los nodos
    x_coords = [v['geometry'].x for v in g_ig.vs]
    y_coords = [v['geometry'].y for v in g_ig.vs]
    coords_layout = list(zip(x_coords, y_coords))

    fig, ax = plt.subplots(figsize=(10, 8))

    # Dibujar capa de cortes si existe
    if gdf_cut is not None:
        gdf_cut.plot(ax=ax, facecolor="none", edgecolor="black")

    ig.plot(
        g_ig,
        target=ax,
        layout=coords_layout,  # <-- usamos las coordenadas reales
        vertex_size=20,
        vertex_color=node_colors,
        edge_color=edge_colors,
        vertex_frame_width=3,
        edge_width=1,
        vertex_label=None,
        margin=20
    )

    # Ajustar límites
    if gdf_cut is not None:
        xmin, ymin, xmax, ymax = gdf_cut.total_bounds
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
    plt.title("Grafo geométrico de "+network)

    # Guardar y abrir
    file_name ="grafo geom "+network+".png"
    plt.savefig(file_name, dpi=1200, bbox_inches="tight")
    plt.close()
    os.startfile(file_name)

network='Energy network'
plot_gdf(network,lines_gdf,nodes_gdf,gdf_cut)
#plot_ig_graph(network, networks_dic[network]['igraph'], gdf_cut, 'red','blue')

In [ ]:
'Plotear NX_GRAPH'
fig, ax = plt.subplots(figsize=(10, 8))
gdf_cut.plot(ax=ax, color="lightgrey", edgecolor="black")
G=g_nx
node_colors = [G.nodes[n].get("color", "red") for n in G.nodes()]

pos = {n: (G.nodes[n]['geometry'].x, G.nodes[n]['geometry'].y) for n in G.nodes()}
nx.draw(G, pos,node_size=0.1,width=0.1, node_color=node_colors,edge_color="green",node_shape=".",linewidths=0, with_labels=False,ax=ax
)

# Guardar imagen
plt.savefig("grafo_nx.png", dpi=1200, bbox_inches="tight")
plt.close()
os.startfile("grafo_nx.png")